<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Ensemble_with_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ensemble


In [25]:
'''imports '''
from google.colab import files
uploaded = files.upload()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from tensorflow.keras.models import Model, load_model


from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%pip install fasttext


# The complete datasets needed
train_df = pd.read_json('train_df_complete.json', orient="records", lines=True)
dev_df = pd.read_json('dev_df_complete.json', orient="records", lines=True)


# The logits from the base model based on the descriptions, on the training and dev set, available in the datasets folder
# on github
#logits_train_model_description = np.load("train_logits_description.npy")
#logits_train_model_features = np.load("train_logits.npy")

# The logits from the base mode based on the extracted features, on the training and dev set , available in datasets folder
# on the github
#logits_dev_model_descriptions = np.load("dev_logits_description.npy")
#logits_dev_model_features = np.load("dev_logits.npy")

Saving dev_df_complete.json to dev_df_complete.json
Saving train_df_complete.json to train_df_complete.json


## Import the 3 models

In [26]:
!pip install huggingface_hub tensorflow --quiet

from huggingface_hub import login, create_repo, upload_folder
import os
import tensorflow as tf
from huggingface_hub import hf_hub_download

# Repo name and model file name
repo_id = "tgarnier067/MNLP-HW1"
model_name_1 = "model_logits_features.keras"
model_name_2 = "model_logits_description.keras"
model_name_3 = "model_logits_photos.keras"

# Download the model from Hugging Face
model_path_1 = hf_hub_download(repo_id=repo_id, filename=model_name_1)
model_path_2 = hf_hub_download(repo_id=repo_id, filename=model_name_2)
model_path_3 = hf_hub_download(repo_id=repo_id, filename=model_name_3)


# Load the .keras model with TensorFlow
model_logits_features = tf.keras.models.load_model(model_path_1)
model_logits_description = tf.keras.models.load_model(model_path_2)
model_logits_photos = tf.keras.models.load_model(model_path_3)

In [ ]:
#from google.colab import files
#uploaded = files.upload()

#from tensorflow.keras.models import load_model
#model_logits_description = load_model("model_logits_description.keras")
#model_logits_features = load_model("model_logits_features.keras")

Saving model_logits_description.keras to model_logits_description.keras
Saving model_logits_features.keras to model_logits_features.keras


## model_logits_description

### Prepare the data

In [3]:
 import fasttext.util
 fasttext.util.download_model('en', if_exists='ignore')  # English
 ft = fasttext.load_model('cc.en.300.bin')

In [27]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')
porter = PorterStemmer()
stop_words = set(stopwords.words('english'))
from sklearn.preprocessing import normalize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [28]:
'''transforming the descriptions to the embeddings space , adding the oov information'''
def transform_description(description):
  oov_words = 0
  description = nltk.word_tokenize(description)
  description = [word for word in description if word not in stop_words]
  description = [word.lower() for word in description]
  for word in description:
    if word not in ft.words:
      oov_words += 1
  description = " ".join(description)
  description_vector = ft.get_sentence_vector(description)
  description_vector = normalize(description_vector.reshape(1, -1), norm='l2')[0]
  description_vector = np.append(description_vector, oov_words)
  return description_vector

In [29]:
train_texts = train_df['description'].to_list()
train_labels = train_df['label_int'].to_list()

# Exctract the text and the label : Dev
dev_texts = dev_df['description'].to_list()
dev_labels = dev_df['label_int'].to_list()

# Vectorize the texts with fasttext
train_vectors = np.array([transform_description(text) for text in train_texts])
dev_vectors = np.array([transform_description(text) for text in dev_texts])

### Apply the model to the data

In [30]:
logits_dev_model_descriptions = model_logits_description.predict(dev_vectors)
logits_train_model_description = model_logits_description.predict(train_vectors)

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step
196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step


In [31]:
df_train_description = pd.DataFrame(logits_train_model_description, columns=['logit_0', 'logit_1', 'logit_2'])
df_train_description['label'] = train_df['label_int']
dev_df_description = pd.DataFrame(logits_dev_model_descriptions, columns=['logit_0', 'logit_1', 'logit_2'])
dev_df_description['label'] = dev_df['label_int']


In [32]:
'''sanity check on the dataframe'''
df_train_description.head(10)

,logit_0,logit_1,logit_2,label
0,1.905099,1.534088,-3.094239,0
1,0.760209,2.901672,-5.288947,1
2,-0.636569,2.368679,-4.468928,1
3,1.309631,1.982280,-3.745397,1
4,-1.129772,2.629756,-4.753025,1
5,1.146501,1.890622,-3.682914,1
6,0.381403,1.764993,-3.796714,0
7,1.686283,2.372820,-4.080890,1
8,-1.380256,2.684189,-4.805523,1
9,1.886965,1.921860,-3.410134,0


## model_logits_features

### Prepare the data

In [33]:
# Get the list of explicatives variables
list_var = ['subclass_depth', 'subclasses', 'instances_of', 'instances_of_up', 'date', 'nb_lang',
       'descr_num_nouns', 'descr_num_verbs',
       'descr_num_adjectives', 'descr_has_location', 'descr_has_ethnic_group',
       'descr_has_event', 'type_cat', 'category_architecture',
       'category_biology', 'category_comics and anime', 'category_fashion',
       'category_films', 'category_food', 'category_geography',
       'category_literature', 'category_media', 'category_music',
       'category_politics', 'category_sports', 'category_transportation',
       'category_visual arts', 'category_combined',
       'main_country_cat'
]

# Create the X and y, train and dev
X_train = train_df[list_var]
y_train = train_df['label_int'].values

X_dev = dev_df[list_var]
y_dev = dev_df['label_int'].values

# Selecting numerical columns from X_train (all of them are int64 or float64)
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Defining pipeline: impute Nan, then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# Appling pipeline to numeric columns
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols)
])


#adjustin the data to proper format
X_train_processed = preprocessor.fit_transform(X_train)
X_dev_processed = preprocessor.transform(X_dev)

# float cast
X_train_processed = X_train_processed.astype('float32')
X_dev_processed = X_dev_processed.astype('float32')

# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=3)
y_dev_cat = to_categorical(y_dev, num_classes=3)

### Apply the model on the data

In [34]:
logits_dev_model_features = model_logits_features.predict(X_dev_processed)
logits_train_model_features = model_logits_features.predict(X_train_processed)

10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 147ms/step
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


## model_logits_photos

In [35]:
train_df['image'] = train_df['image'].apply(lambda x: np.array(x))
dev_df['image'] = dev_df['image'].apply(lambda x: np.array(x))

In [36]:
# Normalisation des images
X_train_fill = np.stack(train_df['image'].to_numpy()).astype('float32') / 255.0
y_train_fill = train_df['label']

X_dev_fill = np.stack(dev_df['image'].to_numpy()).astype('float32') / 255.0
y_dev_fill = dev_df['label']

In [37]:
logits_dev_model_photos = model_logits_photos.predict(X_dev_fill)
logits_train_model_photos = model_logits_photos.predict(X_train_fill)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


## Ensemble model


here we try to combine two classifiers, one basing on the neural network working on features extracted frrom the wikidata/wikipedia, and the second one basing on the neural network working on the embedding representation of the description of the wikipedia/wikidata entities. We combine them with a meta lerner that as training set  takes the logits for each label from both of the trained base neural networks, achieved on the train set. And the validation are the logits of these two networks acquired by predcition on the validation set.

### Build and fit the ensemble model

In [67]:


early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = np.concatenate((logits_train_model_description, logits_train_model_features), axis=1)
train_vectors = np.concatenate((train_vectors, logits_train_model_photos), axis=1)
dev_vectors = np.concatenate((logits_dev_model_descriptions, logits_dev_model_features), axis=1)
dev_vectors = np.concatenate((dev_vectors, logits_dev_model_photos), axis=1)





from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU

#model_Ensemble = models.Sequential()

#model_Ensemble.add(layers.Input(shape=(9,), name='input_layer'))

from tensorflow.keras import regularizers

model_Ensemble = models.Sequential([
    layers.Input(shape=(9,)),
    layers.Dense(16, kernel_regularizer=regularizers.l2(1e-4)),
    LeakyReLU(alpha=0.1),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(8, kernel_regularizer=regularizers.l2(1e-4)),
    LeakyReLU(alpha=0.1),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(3, activation='softmax')
])


model_Ensemble.compile(optimizer=Adam(learning_rate=3e-4),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])


history = model_Ensemble.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.3456 - loss: 1.4492 - val_accuracy: 0.5100 - val_loss: 1.1643 - learning_rate: 3.0000e-04
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4977 - loss: 1.0440 - val_accuracy: 0.6233 - val_loss: 1.0310 - learning_rate: 3.0000e-04
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6219 - loss: 0.8461 - val_accuracy: 0.6667 - val_loss: 0.9576 - learning_rate: 3.0000e-04
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6854 - loss: 0.7374 - val_accuracy: 0.6633 - val_loss: 0.9075 - learning_rate: 3.0000e-04
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7363 - loss: 0.6505 - val_accuracy: 0.6700 - val_loss: 0.8715 - learning_rate: 3.0000e-04
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7909 - loss: 0.5812 - val_accuracy: 0.6833 - val_loss: 0.8410 - learning_rate: 3.0000e-04
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.803

### Evaluation

Seems overfitted to the training set, with discreancy of the accuracies between the training and the validation set.

In [68]:
y_train_pred = model_Ensemble.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)
y_dev_pred = model_Ensemble.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# accu
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)

print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Train accuracy : 93.89%
Dev accuracy   : 72.00%


### Try XGBoost, Logistic Regression and SVM

In [73]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1,
    random_state=42
)

model_xgb.fit(train_vectors, train_labels)
y_pred = model_xgb.predict(dev_vectors)

acc = accuracy_score(dev_labels, y_pred)
print(f"Accuracy XGBoost : {acc:.4f}")

Accuracy XGBoost : 0.6600


In [72]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(max_iter=500, multi_class='multinomial', solver='lbfgs')
model_lr.fit(train_vectors, train_labels)
y_pred = model_lr.predict(dev_vectors)

acc = accuracy_score(dev_labels, y_pred)
print(f"Accuracy Logistic Regression : {acc:.4f}")

Accuracy Logistic Regression : 0.6667


In [71]:
from sklearn.svm import SVC

model_svm = SVC(C=1, kernel='rbf', probability=True)
model_svm.fit(train_vectors, train_labels)
y_pred = model_svm.predict(dev_vectors)

acc = accuracy_score(dev_labels, y_pred)
print(f"Accuracy SVM : {acc:.4f}")

Accuracy SVM : 0.6633


### Exportation of the ensemble model

In [74]:
from huggingface_hub import login, create_repo, upload_folder
import os
import tensorflow as tf
from huggingface_hub import hf_hub_download

login(token="")

repo_name = "MNLP-HW1"
repo_id = f"tgarnier067/{repo_name}"
create_repo(repo_id, exist_ok=True, private=False)
model_folder = "models"
os.makedirs(model_folder, exist_ok=True)

models = {
    "model_ensemble": model_Ensemble
}

# Save the models in separate files
for model_name, model in models.items():
    model_file_path = os.path.join(model_folder, f"{model_name}.keras")
    model.save(model_file_path)

readme_content = """# MNLP HW1 Models
This repository contains multiple models trained for different tasks. Each model is saved as a separate `.keras` file.
"""

with open(os.path.join(model_folder, "README.md"), "w") as f:
    f.write(readme_content)

upload_folder(
    repo_id=repo_id,
    folder_path=model_folder,
    path_in_repo=""
)

print(f"All models have been successfully uploaded to Hugging Face in the repo {repo_id}!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/hf_api.py:9561: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


All models have been successfully uploaded to Hugging Face in the repo tgarnier067/MNLP-HW1!


# CNN model only

In [41]:
early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = logits_train_model_photos
dev_vectors = logits_dev_model_photos



from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU

model_photos = models.Sequential()

model_photos.add(layers.Input(shape=(3,), name='input_layer'))

model_photos.add(layers.Dense(3, activation='softmax'))

model_photos.compile(optimizer=Adam(learning_rate=1e-3),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])



history = model_photos.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.4200 - loss: 4.2080 - val_accuracy: 0.4567 - val_loss: 3.5064 - learning_rate: 0.0010
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4325 - loss: 3.3012 - val_accuracy: 0.4467 - val_loss: 3.0949 - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4541 - loss: 2.7731 - val_accuracy: 0.4533 - val_loss: 2.6887 - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4767 - loss: 2.0359 - val_accuracy: 0.4600 - val_loss: 2.3034 - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4844 - loss: 1.4612 - val_accuracy: 0.4533 - val_loss: 1.9469 - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5203 - loss: 0.9706 - val_accuracy: 0.4767 - val_loss: 1.7570 - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6102 - loss: 0.7740 - val_ac

In [43]:
y_train_pred = model_photos.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)
y_dev_pred = model_photos.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# accu
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)

print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Train accuracy : 65.93%
Dev accuracy   : 50.00%


# Description model only

In [46]:
early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = logits_train_model_description
dev_vectors = logits_dev_model_descriptions



from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU

model_description = models.Sequential()

model_description.add(layers.Input(shape=(3,), name='input_layer'))

model_description.add(layers.Dense(3, activation='softmax'))

model_description.compile(optimizer=Adam(learning_rate=1e-3),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])



history = model_description.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.4975 - loss: 2.0145 - val_accuracy: 0.6067 - val_loss: 1.1828 - learning_rate: 0.0010
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4917 - loss: 1.5174 - val_accuracy: 0.6233 - val_loss: 0.9724 - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5032 - loss: 1.0860 - val_accuracy: 0.6300 - val_loss: 0.8366 - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5661 - loss: 0.7911 - val_accuracy: 0.6667 - val_loss: 0.7782 - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6846 - loss: 0.6225 - val_accuracy: 0.6933 - val_loss: 0.7666 - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7489 - loss: 0.5329 - val_accuracy: 0.6967 - val_loss: 0.7724 - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7763 - loss: 0.4780 - val_a

In [47]:
y_train_pred = model_description.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)
y_dev_pred = model_description.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# accu
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)

print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Train accuracy : 76.98%
Dev accuracy   : 69.67%


# Features model only

In [48]:
early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = logits_train_model_features
dev_vectors = logits_dev_model_features



from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU

model_features = models.Sequential()

model_features.add(layers.Input(shape=(3,), name='input_layer'))

model_features.add(layers.Dense(3, activation='softmax'))

model_features.compile(optimizer=Adam(learning_rate=1e-3),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])



history = model_features.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7750 - loss: 0.5525 - val_accuracy: 0.7500 - val_loss: 0.6341 - learning_rate: 0.0010
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7719 - loss: 0.5636 - val_accuracy: 0.7500 - val_loss: 0.6340 - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7675 - loss: 0.5631 - val_accuracy: 0.7500 - val_loss: 0.6334 - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7711 - loss: 0.5533 - val_accuracy: 0.7500 - val_loss: 0.6331 - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7760 - loss: 0.5488 - val_accuracy: 0.7500 - val_loss: 0.6330 - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7744 - loss: 0.5415 - val_accuracy: 0.7533 - val_loss: 0.6319 - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7800 - loss: 0.5405 - val_ac

In [49]:
y_train_pred = model_features.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)
y_dev_pred = model_features.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# accu
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)

print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Train accuracy : 77.01%
Dev accuracy   : 75.33%
